In [15]:
# Importación de bibliotecas necesarias
import cv2                          # OpenCV para procesamiento de imágenes
import numpy as np                 # NumPy para operaciones numéricas
from PIL import Image, ImageDraw, ImageFont  # Para crear y editar imágenes tipo GIF

# 1. Cargar imagen en escala de grises y color
img_gray = cv2.imread('monedas.jpg', cv2.IMREAD_GRAYSCALE)  # Imagen en escala de grises para segmentación
img_color = cv2.imread('monedas.jpg')                        # Imagen original en BGR (formato de OpenCV)
img_rgb = cv2.cvtColor(img_color, cv2.COLOR_BGR2RGB)         # Conversión a RGB para visualización y PIL

# 2. Aplicar suavizado + umbral adaptativo
# Se aplica un desenfoque Gaussiano para reducir el ruido antes de segmentar
blurred = cv2.GaussianBlur(img_gray, (11, 11), 0)

# Umbral adaptativo inverso: monedas oscuras sobre fondo claro
thresh = cv2.adaptiveThreshold(
    blurred, 255,                            # Máximo valor
    cv2.ADAPTIVE_THRESH_GAUSSIAN_C,         # Método adaptativo
    cv2.THRESH_BINARY_INV,                  # Inverso: monedas = blanco
    11, 2                                   # Bloque de 11x11 y constante substraída = 2
)

# 3. Detección de contornos
contours, _ = cv2.findContours(
    thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
)  # Detecta contornos externos

img_result = img_rgb.copy()  # Copia de la imagen para dibujar resultados
areas, perimetros = [], []  # Listas para almacenar métricas

# Iterar sobre cada contorno
for cnt in contours:
    area = cv2.contourArea(cnt)
    if area < 1000:
        continue  # Ignorar contornos pequeños (ruido)

    perim = cv2.arcLength(cnt, True)
    areas.append(area)
    perimetros.append(perim)

    # Calcular bounding box y dibujarla
    x, y, w, h = cv2.boundingRect(cnt)
    cv2.rectangle(img_result, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # Calcular el centro de masa del contorno
    M = cv2.moments(cnt)
    if M["m00"] != 0:
        cx = int(M["m10"] / M["m00"])
        cy = int(M["m01"] / M["m00"])
        cv2.circle(img_result, (cx, cy), 5, (255, 0, 0), -1)

    # Dibujar el contorno sobre la imagen
    cv2.drawContours(img_result, [cnt], -1, (255, 255, 0), 2)

# 4. Preparar frames para el GIF
frames = []

# Convertir imágenes a formato PIL
original_pil = Image.fromarray(img_rgb)
thresh_rgb = cv2.cvtColor(thresh, cv2.COLOR_GRAY2RGB)  # Convertir binaria a RGB para compatibilidad con PIL
thresh_pil = Image.fromarray(thresh_rgb)
result_pil = Image.fromarray(img_result)

# Función auxiliar para agregar etiquetas con borde
def add_label(pil_img, label_text):
    img = pil_img.copy()
    draw = ImageDraw.Draw(img)
    try:
        font = ImageFont.truetype("arial.ttf", 32)
    except:
        font = ImageFont.load_default()

    x, y = 10, 10
    # Dibujar contorno negro del texto
    for dx in [-1, 0, 1]:
        for dy in [-1, 0, 1]:
            if dx != 0 or dy != 0:
                draw.text((x + dx, y + dy), label_text, font=font, fill=(0, 0, 0))
    # Dibujar texto blanco encima
    draw.text((x, y), label_text, font=font, fill=(255, 255, 255))
    return img

# Crear los 3 fotogramas del GIF con sus etiquetas
frames.append(add_label(original_pil, "Imagen original"))
frames.append(add_label(thresh_pil, "Segmentación binaria"))
frames.append(add_label(result_pil, "Contornos y detección"))

# 5. Guardar GIF animado
frames[0].save(
    "deteccion_monedas.gif",
    save_all=True,
    append_images=frames[1:],  # Agrega los siguientes frames
    duration=1000,             # 1 segundo por fotograma
    loop=0                     # Repetir infinitamente
)

print("✅ GIF guardado como 'deteccion_monedas.gif'")


✅ GIF guardado como 'deteccion_monedas.gif'
